In [ ]:
%run TFIDF_Converter.ipynb

In [ ]:
def initializeDocVectors(docVectorDir):
    docVectorList = os.listdir(docVectorDir)
    corpusDict = dict() # key=docID, val=docVector
    for doc in docVectorList:
        docVector = dict()
        
        # ignore macOS filesystem file and jupyter files and dictionary from create term vectors
        if doc != ".DS_Store" and doc != ".ipynb_checkpoints" and doc != "dictionary.txt":
            with open(docVectorDir+doc, 'r') as f:
                for line in f.readlines():
                    line = line.strip()
                    items = line.split(' ')
                    termID = items[0]
                    score = items[1] # term's tf-idf score
                    docVector[termID] = score
                f.close()
            docID = int(doc.split('.')[0])
            corpusDict[docID] = docVector
    return corpusDict

In [ ]:
def hierachicalClustering(corpusDict, requestClusterNumber):
    
    # because docID index from 1, thus insert a empty element in the begining stand for 0
    docList = [None]+sorted(list(corpusDict.keys()))
    docSize = len(docList)
    availableMergedIndicator = [True for i in range(docSize)] # if the candidate doc available
    candidateList = list()
    docScoreMatrix = [[0 for col in range(docSize)] for row in range(docSize)] # simiarity matrix
    print("Start Calculate Score Matrix", datetime.datetime.now())
    
    for i in range(1, docSize):
        for j in range(i+1, docSize):
            cosSim = cosine(docVectorDir+str(docList[i])+'.txt', docVectorDir+str(docList[j])+'.txt')
            memSet = {docList[i], docList[j]} # memberSet
            score = (cosSim, memSet) # sim score get from memberSet{i, j}
            candidateList.append(score)
            docScoreMatrix[i][j] = cosSim # update score matrix
            docScoreMatrix[j][i] = cosSim

    print("Finish Calculate Score Matrix", datetime.datetime.now())

    
    print("Start Clustering", datetime.datetime.now())
    genID = 0
    
    # record clustering situation in every iteration
    intermediateClusterPool = dict()  # key=generated Iteration, value=clusterMembers
    
    # record available candidate now  ,  k=idx, v=docMembers
    availableCandidateMembersIndex_dict = {d: {d} for d in range(1, docSize)} 
    
    for m in range(1, docSize):
        if(not candidateList): break
        # gen a new cluster (i.e., highest score)
        candidateList.sort(key=lambda x: x[0], reverse=True)
        print("Generate New Cluster:", datetime.datetime.now()) # check time
        bestPair = candidateList[0]
        scoreOfBest = bestPair[0]
        memSet = bestPair[1]
        
        # delete unavailable candidate
        for mem in memSet:
            availableMergedIndicator[mem] = False
            if mem in availableCandidateMembersIndex_dict.keys():
                del availableCandidateMembersIndex_dict[mem]
        
        shadowCandidateList = candidateList[:]

        for candidate in shadowCandidateList:
            memOfPair = candidate[1]
            if not memOfPair.isdisjoint(memSet):
                candidateList.remove(candidate)
                
        # get the rep Index of temp Cluster
        repId = memSet.pop()
        memSet.add(repId)
        
        # update the representative members(e.g., 1={1,2,3}; 6={6,7,8})
        availableCandidateMembersIndex_dict[repId] = memSet.copy()
        availableMergedIndicator[repId] = True
        
        # output cluster
        intermediateClusterPool[genID] = availableCandidateMembersIndex_dict.copy()
        if(len(availableCandidateMembersIndex_dict.values()) == requestClusterNumber):
            break

        # update similarity score matrix with new candidate.
        for n in range(1, docSize):
            if(availableMergedIndicator[n] is True) and (n not in memSet):

                # Complete-Link: find closest docVector
                newSim = 1
                for mem in memSet:
                    if docScoreMatrix[n][mem] <= newSim:
                        newSim = docScoreMatrix[n][mem]

                docScoreMatrix[n][repId] = newSim # update pair score in matrix
                docScoreMatrix[repId][n] = newSim
                
                newMemSet = memSet.copy()
                memberOfN = availableCandidateMembersIndex_dict[n]
                newMemSet.update(memberOfN)
                newCandidate = (newSim, newMemSet)
                candidateList.append(newCandidate)
        genID += 1
    return intermediateClusterPool